In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer

# 加载与模型对应的 tokenizer
model_id = "meta-llama/Meta-Llama-3-8B"  # 或者你使用的 llama3 模型名
tokenizer = AutoTokenizer.from_pretrained(model_id)


dataset = load_dataset("cmrc2018")
train_data = dataset['train']

def format_qa(example):
    prompt = f"问：{example['question']}\n上下文：{example['context']}\n答："
    answer = example['answers']['text'][0] if example['answers']['text'] else "无答案"
    return {
        "input_ids": tokenizer(prompt, truncation=True)["input_ids"],
        "labels": tokenizer(answer, truncation=True)["input_ids"]
    }

train_data = train_data.map(format_qa)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id, device_map=, low_cpu_mem_usage=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": "cpu"},
    torch_dtype=torch.float16
)
peft_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05, task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="./lora-llama3-zhqa",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

trainer.train()


TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'tdevice_map'